# # Récupération de la donnée "arrets-de-transport" de data.ampmetropole.fr à partir d'un bbox
https://trouver.datasud.fr/dataset/sync-fr-gtfs-stop-connectehttpsdataampmetropolefrexplore/resource/c23ac74c-a621-41e7-9b41-6218712dcd35

In [5]:
#exemple : bbox = (S=43.295426, N=43.317027, E=5.389065, W=5.411281)
import requests
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, box, polygon
from ipyleaflet import Map, GeoJSON, GeoData, ZoomControl

# URL du fichier geojson contenant les arrêts de transport en data.ampmetropole.fr
url_geojson = 'https://data.ampmetropole.fr/api/v2/catalog/datasets/fr-gtfs-stop-connecte/exports/geojson'
r =  requests.get(url_geojson)
dictjson=r.json()

# Créer un GeoDataFrame à partir du dictionnaire du geojson
gdf_arrets_datasud = gpd.GeoDataFrame.from_features(dictjson)

E=5.389065
S=43.295426
W=5.411281
N=43.317027

bound=box(E,S,W,N)
boundgdf = gpd.GeoDataFrame({'geometry': [bound]}, crs='EPSG:4326') #avoir le geodataframe du bbox

bounding_area = boundgdf
boundgdf.set_crs(epsg=4326, inplace=True)
gdf_arrets_datasud.set_crs(epsg=4326, inplace=True)

gdf_filtered = gpd.sjoin(gdf_arrets_datasud, bounding_area, predicate='intersects')
gdf_filtered.set_crs(4326, allow_override=True)
print("il s'agit de %d arrets dee transport au niveau du territoire d'étude" % len(gdf_filtered))
#arrets_bbox = gdf_filtered.to_file(output_folder)

bbox=(S,N,E,W)
xmoy=(bbox[2]+bbox[3])/2
ymoy=(bbox[0]+bbox[1])/2

m = Map(center=(ymoy, xmoy), zoom=14)

gdf =  gdf_filtered
bounding = boundgdf

geo_data1 = GeoData(geo_dataframe = gdf,
    style={'color': 'black', 'radius':3, 'fillColor': '#cc3333', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':1},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'Release')

geo_data2 = GeoData(geo_dataframe = bounding,
    style={'color': 'blue', 'radius':2, 'fillColor': '#004dff', 'opacity':1, 'weight':2, 'dashArray':'1', 'fillOpacity':0})

m.add(geo_data2)
m.add(geo_data1)
print("end")
display(m)

il s'agit de 143 arrets dee transport au niveau du territoire d'étude
end


Map(center=[43.3062265, 5.400173000000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

In [7]:
import requests
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, box, polygon

(S, N, E, W) = (43.295426, 43.317027, 5.389065, 5.411281)
output_folder = "../data/processed/arrets_bbox_datasud.gpkg"
url_geojson = 'https://data.ampmetropole.fr/api/v2/catalog/datasets/fr-gtfs-stop-connecte/exports/geojson'

def get_arretsdetransport_datasud(url):
    # Lire le fichier geojson
    r =  requests.get(url)
    dictjson=r.json()
    # Créer un GeoDataFrame à partir du dictionnaire du geojson
    gdf_arrets_datasud = gpd.GeoDataFrame.from_features(dictjson)
    return(gdf_arrets_datasud)
    
def get_arrets_BBOX_datasud(url, S, N , E, W, output_folder) :
    gdf_arrets_datasud = get_arretsdetransport_datasud(url)

    bound=box(E,S,W,N)
    boundgdf = gpd.GeoDataFrame({'geometry': [bound]}, crs='EPSG:4326') #avoir le geodataframe du bbox
    
    bounding_area = boundgdf
    boundgdf.set_crs(epsg=4326, inplace=True)
    gdf_arrets_datasud.set_crs(epsg=4326, inplace=True)
    
    gdf_filtered = gpd.sjoin(gdf_arrets_datasud, bounding_area, predicate='intersects')
    gdf_filtered.set_crs(4326, allow_override=True)
    print("il s'agit de %d arrets dee transport au niveau du territoire d'étude" % len(gdf_filtered))
    arrets_bbox_gpkg = gdf_filtered.to_file(output_folder)
    return(gdf_filtered)

get_arrets_BBOX_datasud(url_geojson, S, N , E, W, output_folder)
print("end")

il s'agit de 143 arrets dee transport au niveau du territoire d'étude
end
